# Collection of Twitter data for the <font color='red'>#Day-zero</font> sentiment ananlysis
Data is scraped from twitter relating to the waer  crisis issue in Cape Town

- drought sort of began in 2015 in the Western cape
- Cape town Details:
    - -33.9249° S, 18.4241° E
- Countries we do business in:


### Loading Packages

In [6]:
import numpy as np
import pandas as pd
import tweepy as tpy
import sqlite3
import json
# import jsonpickle

### Authotrization

In [2]:
# import access keys
from credentials import *

# API setup
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tpy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    # Return API with authentication:
    api = tpy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)  # todo: include rate limit parameters
    return api

api = twitter_setup()

In [17]:
tweet = api.home_timeline(count=1)[0]

In [40]:
[tweet.id, tweet.created_at, tweet.text, tweet.user.followers_count, tweet.user.verified, tweet.possibly_sensitive, 
 tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['hashtags'], tweet.sc ]

[956781542491283456,
 datetime.datetime(2018, 1, 26, 6, 51, 10),
 'RT @RickyPo: Sub-Saharan nations lead the way in open access https://t.co/c7XgMmIZsp',
 684,
 False,
 False,
 None,
 None,
 1,
 0,
 []]

In [70]:
# api.geo_search
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1517096073}}

After perusing a few tweets as well as news articles, the following keywords have been suggested

In [177]:
search_query = ('"day zero" water OR water crisis cape OR "day zero" crisis OR cape water OR cape drought')
max_tweets= 100000
tweets_per_query = 100

In [178]:
#with open('../data/tweets.json', mode='w') as f:  # do a check if file already exists as 'x' creates file each time
date = []
text = []
retweet_count = []
follower_count = []
location = []
tags = []
verified = []

tweets = tpy.Cursor(api.search, q=search_query).items(max_tweets)

for tweet in tweets:
    if hasatt
    text.append(tweet.text)
    date.append(tweet.created_at)
    retweet_count.append(tweet.retweet_count)
    follower_count.append(tweet.user.followers_count)
    location.append(tweet.user.location)
    tags.append(tweet.entities['hashtags'])
    verified.append(tweet.user.verified)
    
tweet_data = pd.DataFrame({'text': text,
                          'date': date,
                          'retweet_count': retweet_count,
                          'follower_count': follower_count,
                          'location': location,
                          'tags': tags,
                          'verified': verified})

,date,follower_count,location,retweet_count,tags,text,verified
0,2018-01-26 09:47:59,247,UK,0,"[{'text': 'water', 'indices': [62, 68]}]","Another commodity, alongside food, that we tak...",False
1,2018-01-26 09:47:59,737,Washington (the state),1,"[{'text': 'CapeTown', 'indices': [21, 30]}]",RT @TheDailyClimate: #CapeTown authorities are...,False
2,2018-01-26 09:45:10,15189,"Bozeman, Mont.",1,"[{'text': 'CapeTown', 'indices': [0, 9]}]",#CapeTown authorities are scrambling to persua...,False
3,2018-01-26 02:14:37,650,"Cape Town, South Africa",147,[],"RT @AssaadRazzouk: Day Zero: Within 4 Months, ...",False
4,2018-01-26 00:00:00,419,Japan,134,[],RT @Reuters: WATCH: Cape Town's authorities ar...,False


In [ ]:
# todo kparse location column nicely

def parse_location(string):
    
    """
    takes in a string and returns the best guess for a country name
    """

In [41]:
import sqlite3

conn = sqlite3.connect('twitter.db')
c = conn.cursor()

c.execute('''CREATE TABLE tweets
    (tweetText text,
    user text,
    followers integer,
    date text,
    location text)''')
conn.commit()
conn.close()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 18))



OperationalError: table tweets already exists

In [72]:
# DB stuff
conn = sqlite3.connect('twitter.db')
c = conn.cursor()

class Tweet():
    
    def __init__(self, text, user, followers, date, location):
        self.text = text
        self.user = user
        self.followers = followers
        self.date = date
        self.location = location

        
# Inserting that data into the DB
    def insertTweet(self):

        c.execute("INSERT INTO tweets (tweetText, user, followers, date, location) VALUES (?, ?, ?, ?, ?)",
            (self.text, self.user, self.followers, self.date, self.location))
        conn.commit()

In [73]:
class TweetStreamListener(tpy.StreamListener):
    
    def on_data(self, data):
        
        try:
            
            # make it json
            tweet = json.loads(data)
            
            # filter
            if not tweet['retweeted'] and 'RT @' not in tweet['text']:
                
                # get user via tweepy so we can get their no of followers
                user_profile = api.get_user(tweet['user']['screen_name'])
                
                # create tweet instance with data
                tweet_data = Tweet(
                    str(tweet['text'].encode('utf-8')),
                    tweet['user']['screen_name'],
                    user_profile.followers_count,
                    tweet['created_at'],
                    tweet['user']['location'])

                tweet_data.insertTweet()
                print("success")
                
        except Exception as e:
            print(e)
            pass
        
        return True

In [74]:
if __name__ == '__main__':

    # Run the stream
    l = TweetStreamListener()
    stream = tpy.Stream(api.auth, l)

    # Filter the stream for these keywords. Add whatever you want here!
    stream.filter(track=['day zero water', 'water crisis cape', 'rain cape day zero', 'cape town drought'])

success
success
success


KeyboardInterrupt: 

In [75]:
conn.close()

In [18]:
conn = sqlite3.connect('example.db')

In [19]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [28]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)
conn.commit()
conn.close()

In [ ]:
conn.close()

In [29]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [104]:
conn = sqlite3.connect('test.db')
c = conn.cursor()

for row in c.execute('SELECT * FROM tweets'):
        print(row)

("b'Man it\\xe2\\x80\\x99s nothing wrong with having a type. If you not his type your just not his type. And yes you should be who yo\\xe2\\x80\\xa6 https://t.co/egNENim04q'", 'MR_B_MARELL', 510, 'Sat Jan 27 22:47:08 +0000 2018', 'Camden, NJ')
('b"@Kiteh_Kawasaki Here\'s mine: During this 28 day period, you earned 4.9K impressions per day. That\'s 137K. Hmmm I se\\xe2\\x80\\xa6 https://t.co/EN0hFUSWcy"', 'Spaneele', 267, 'Sat Jan 27 22:47:31 +0000 2018', 'United States')
("b'April 12\\xe2\\x80\\xa6 https://t.co/qyc917vS1E'", 'mg5904', 75, 'Sat Jan 27 22:49:25 +0000 2018', 'Gallupville, USA')
("b'Day 1 of living zero waste! Was doing great rejecting straws and baggies but got a muffin with a wrapper around it\\xe2\\x80\\xa6 https://t.co/6XmYItKC4i'", 'laurenshelburne', 516, 'Sat Jan 27 22:50:54 +0000 2018', 'New York, NY')
("b'Stunning day @ Piha Beach. The goal today is zero rescues. Tough call. https://t.co/ZLFlPlK6NP'", 'BRJDoyle', 17, 'Sat Jan 27 22:51:37 +0000 2018', 'New Zealand')

In [98]:
def delete_all_tasks(db, table):
    """
    Delete all rows in the tasks table
    :param conn: Connection to the SQLite database
    :return:
    """
    conn = sqlite3.connect(db)
    sql = 'DELETE FROM '+ table
    display(sql)
    cur = conn.cursor()
    cur.execute(sql)
delete_all_tasks('twitter.db', table='tweets')

'DELETE FROM tweets'

In [102]:
conn = sqlite3.connect('twitter.db')
sql = 'DELETE FROM tweets'
cur = conn.cursor()
cur.execute(sql)
conn.commit()
conn.close()

OperationalError: database is locked